# Pandas Filtering and Sorting


Import `pandas` and `numpy` to follow along with the examples in this page.


In [141]:
import pandas as pd
import numpy as np

---

## 🧮 Filtering `Series`


To filter rows in a `DataFrame` or `Series`, we can use boolean indexing. This involves creating a boolean mask that indicates which rows meet a certain condition, and then using that mask to select the desired rows.

Boolean indexing can be applied to both `DataFrame` and `Series` objects in pandas. This is a powerful and flexible technique for filtering data based on specific criteria.

We'll start with a simple example using a `Series`.

▶️ Create a `Series` named `nums` with the following four integers: `-20`, `-10`, `10`, `20`.


In [142]:
nums = pd.Series([-20, -10, 10, 20])

display(nums)

0   -20
1   -10
2    10
3    20
dtype: int64

👉 Our goal is to filter the `Series` so that it only contains **positive** values. We'll first manually create a boolean mask that indicates which values are positive.

▶️ Create a new `Series` named `keep` with the following four boolean values: `False`, `False`, `True`, `True`.


In [143]:
keep = pd.Series([False, False, True, True])

display(keep)

0    False
1    False
2     True
3     True
dtype: bool

Let's visualize the two `Series` (`nums` and `keep`) you've created.

![nums-and-keep](https://github.com/bdi475/images/blob/main/nums-and-keep-series.png?raw=true)


Once we have a boolean mask (the `keep` `Series`), we can use it to filter the original `Series` (`nums`).


In [144]:
nums[keep]

2    10
3    20
dtype: int64

This can be confusing at first, but the idea is that the `True` values in the `keep` `Series` indicate which values in the `nums` `Series` should be kept. The `False` values indicate which values should be discarded. This is done by passing the `keep` `Series` inside the square brackets of the `nums` `Series`, like this: `nums[keep]`.

This is called boolean indexing, and it's a powerful way to filter data in pandas. The boolean `Series` is a boolean _mask_ that tells pandas which values to keep and which to discard.

If you're confused about what just happened, this visualization may give you a better idea.

![nums-and-keep-filter-result](https://github.com/bdi475/images/blob/main/nums-and-keep-filter-result.png?raw=true)


:::{danger} Same Length Requirement
When using boolean indexing, make sure the boolean mask (`keep`) has the same length as the original `Series` (`nums`). Otherwise, you'll encounter an error.

For example, if `keep` has fewer elements than `nums`, you'll get an `IndexingError`. In the example below, `keep_incorrect` has only three boolean values, while `nums` has four integers. This raises an `IndexingError`.

⛔️ **Heads-up**: The code below will throw an error.

```python
nums = pd.Series([-20, -10, 10, 20])
keep_incorrect = pd.Series([False, False, True])
nums[keep_incorrect]
# IndexingError: Unalignable boolean Series provided as indexer
```

:::


### 🗂️ Creating Boolean Masks Programmatically

The previous example was a bit contrived. We manually created a boolean mask (`keep`) to filter the `nums` `Series`. This is not how you would do it in practice. If you had a `Series` with millions of elements, you wouldn't want to manually create a boolean mask with millions of `True` and `False` values.

Is there a way to create the boolean mask _programmatically_? Yes! You can use comparison operators to create boolean masks.


In [145]:
keep_by_comparison = nums > 0

keep_by_comparison

0    False
1    False
2     True
3     True
dtype: bool

The expression `nums > 0` compares each element in the `nums` `Series` to `0`, returning a new boolean `Series` where each value indicates whether the corresponding value in `nums` is greater than `0`.

You can use other comparison operators as well, such as `<`, `<=`, `>=`, `==`, and `!=`.

The resulting boolean `Series` is identical to the manually created `keep` `Series` from earlier. We can check this using the `pd.testing.assert_series_equal()` function, which raises an error if the two `Series` are not equal.


In [146]:
# Manually created boolean mask
display(keep)

# Programmatically created boolean mask
display(keep_by_comparison)

# Check that both boolean masks are identical
pd.testing.assert_series_equal(keep, keep_by_comparison)

0    False
1    False
2     True
3     True
dtype: bool

0    False
1    False
2     True
3     True
dtype: bool

We can use the `keep_by_comparison` to filter positive values in `nums`. This works the same way as before where the `True` values in the boolean mask indicate which values to keep. `False` values indicate which values to discard.


In [147]:
nums[keep_by_comparison]

2    10
3    20
dtype: int64

:::{tip} Boolean Indexing Does Not Modify Original Data
Filtering elements using a boolean mask returns **a new `Series`** without modifying the original `Series`.

```python
print("Negative Values (filtered):")
display(nums[nums < 10])

# Output:
# Negative Values (filtered):
# 0   -20
# 1   -10
# dtype: int64

print("\n\nOriginal Values:")
display(nums)

# Output:
# Original Values:
# 0   -20
# 1   -10
# 2    10
# 3    20
# dtype: int64
```

Notice how the original `nums` `Series` remains unchanged after filtering.
:::


---

**🎯 Example: Filter even numbers**


In [148]:
all_nums = pd.Series([2, 5, 4, 8, -2, -5, -11, 13, 4])

even_nums = all_nums[all_nums % 2 == 0]

even_nums

0    2
2    4
3    8
4   -2
8    4
dtype: int64

---

## 🧩 Filtering a `DataFrame`


Recall that a `DataFrame` is a collection of one or more columns, where each column is a `Series`. This means that filtering a `DataFrame` is very similar to filtering a `Series`.

We'll start by creating a simple `DataFrame` that contains a list of transactions.


In [149]:
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)

display(df)

,name,amount
0,John,-20
1,Mary,-10
2,Tom,10
3,John,20


Assume we want to filter the `DataFrame` so that it only contains transactions made by `'John'`. There are two transactions made by `'John'`. We can do this by manually creating a boolean mask that indicates which rows have the `name` equal to `'John'`.


In [150]:
is_john = pd.Series([True, False, False, True])

is_john

0     True
1    False
2    False
3     True
dtype: bool

As before, the `True` values in the boolean mask indicate which rows to keep. The `False` values indicate which rows to discard. This is done by passing the boolean mask inside the square brackets of the `DataFrame`, like this: `df[is_john]`.


In [151]:
result = df[is_john]

result

,name,amount
0,John,-20
3,John,20


The visualization below illustrates how `df[is_john]` works.

![mini-dataframe-filter-rows](https://github.com/bdi475/images/blob/main/filter-mini-dataframe-result.png?raw=true)

&nbsp;


**🎯 Example: Find all positive transactions programmatically**


In [152]:
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)
df

,name,amount
0,John,-20
1,Mary,-10
2,Tom,10
3,John,20


To only keep rows where the transaction amount was positive, we can programmatically create a boolean mask that indicates which rows have the `amount` greater than `0`. We can then use this boolean mask to filter the `DataFrame`.


In [153]:
df_pos = df[df["amount"] > 0]

df_pos

,name,amount
2,Tom,10
3,John,20


:::{note} What does `df["amount"] > 0` do?
The expression `df["amount"] > 0` compares each element in the `amount` column of the `DataFrame` to `0`, returning a new boolean `Series` where each value indicates whether the corresponding value in the `amount` column is greater than `0`.

Recall that a column in a `DataFrame` is a `Series`. So `df["amount"]` returns the `amount` column as a `Series`. The comparison operator `>` is then applied element-wise to each value in the `amount` `Series`.

This is similar to how we created a boolean mask for the `nums` `Series` earlier. Here is the code again for reference:

```python
# Create the nums Series
nums = pd.Series([-20, -10, 10, 20])
# Create a boolean mask to keep only positive values
keep_by_comparison = nums > 0

# Create the DataFrame with transactions
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)
# Create a boolean mask to keep only positive transactions
keep_positive_transactions = df["amount"] > 0
```

:::


---

## ⚙️ Logical Operators in `pandas`


Pandas supports logical operations on boolean `Series`. This allows you to combine multiple conditions to create more complex boolean masks.

There are three logical operations you can perform on boolean `Series`:

- `&`: Logical **AND**
- `|`: Logical **OR**
- `~`: Logical **NOT**

These operators perform element-wise logical operations on the boolean `Series`.

For more examples, you can refer to the [Pandas documentation on indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing).


### 🤝 Logical AND

A logical AND operator `&` returns `True` only if both the operands are `True`.

![s1_AND_s2](https://github.com/bdi475/images/blob/main/s1-AND-s2.png?raw=true)


▶️ Perform a logical AND operation (`&`) on `s1` and `s2` and store the result to a new variable named `s1_AND_s2`.


In [154]:
s1 = pd.Series([True, True, False, False])
s2 = pd.Series([True, False, True, False])

s1_AND_s2 = s1 & s2
s1_AND_s2

0     True
1    False
2    False
3    False
dtype: bool

In [155]:
# Display s1, s2, s1_AND_S2 together as a DataFrame
pd.DataFrame({"s1": s1, "s2": s2, "s1_AND_s2": s1_AND_s2})

,s1,s2,s1_AND_s2
0,True,True,True
1,True,False,False
2,False,True,False
3,False,False,False


### 📍 Logical OR

A logical OR operator `|` returns `True` if either of the operands is `True`.

![s1_OR_s2](https://github.com/bdi475/images/blob/main/s1-OR-s2.png?raw=true)


▶️ Perform a logical OR operation (`|`) on `s1` and `s2` and store the result to a new variable named `s1_OR_s2`.


In [156]:
s1 = pd.Series([True, True, False, False])
s2 = pd.Series([True, False, True, False])

s1_OR_s2 = s1 | s2
s1_OR_s2

0     True
1     True
2     True
3    False
dtype: bool

In [157]:
# Display s1, s2, s1_OR_s2 together as a DataFrame
pd.DataFrame({"s1": s1, "s2": s2, "s1_OR_s2": s1_OR_s2})

,s1,s2,s1_OR_s2
0,True,True,True
1,True,False,True
2,False,True,True
3,False,False,False


### ❗ Logical NOT

A logical NOT operator `~` reverses each operand.

![NOT_s1](https://github.com/bdi475/images/blob/main/NOT-s1.png?raw=true)


▶️ Perform a logical OR operation (`~`) on `s1` and store the result to a new variable named `NOT_s1`.


In [158]:
s1 = pd.Series([True, True, False, False])

NOT_s1 = ~s1
NOT_s1

0    False
1    False
2     True
3     True
dtype: bool

In [159]:
# Display s1 and NOT_s1 together as a DataFrame
pd.DataFrame({"s1": s1, "NOT_s1": NOT_s1})

,s1,NOT_s1
0,True,False
1,True,False
2,False,True
3,False,True


### ✅ Putting It All Together

Here is a summary of the logical operations you've performed so far.


In [160]:
pd.DataFrame(
    {"s1": s1, "s2": s2, "s1_AND_s2": s1_AND_s2, "s1_OR_s2": s1_OR_s2, "NOT_s1": NOT_s1}
)

,s1,s2,s1_AND_s2,s1_OR_s2,NOT_s1
0,True,True,True,True,False
1,True,False,False,True,False
2,False,True,False,True,True
3,False,False,False,False,True


**🎯 Example: Find John's positive transaction(s)**


In [161]:
# DO NOT CHANGE THE CODE IN THIS CELL
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)

# Create two boolean masks
is_john = df["name"] == "John"
is_positive = df["amount"] > 0

# Apply both conditions using the & operator
df_john_and_pos = df[is_john & is_positive]

df_john_and_pos

,name,amount
3,John,20


:::{tip} What do `is_john` and `is_positive` represent?

- `is_john` is a boolean mask that indicates which rows in the `DataFrame`
  have the name "John".
- `is_positive` is a boolean mask that indicates which rows in the `DataFrame`
  have a positive `amount`.

```python
is_john
display(is_john)

# Output:
# 0     True
# 1    False
# 2    False
# 3     True
# Name: name, dtype: bool

is_positive
display(is_positive)
# Output:
# 0    False
# 1    False
# 2     True
# 3     True
# Name: amount, dtype: bool
```

:::


Here's a visualization to help you understand what `is_john & is_positive` does.

![is_john_AND_is_positive](https://github.com/bdi475/images/blob/main/is-john-AND-is-positive.png?raw=true)


---

**🎯 Example: Find transactions that are made by John OR are positive**


In [162]:
# DO NOT CHANGE THE CODE IN THIS CELL
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)
df

,name,amount
0,John,-20
1,Mary,-10
2,Tom,10
3,John,20


In [163]:
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)

# Create two boolean masks
is_john = df["name"] == "John"
is_positive = df["amount"] > 0

# Apply either condition using the | operator
df_john_or_pos = df[is_john | is_positive]

df_john_or_pos

,name,amount
0,John,-20
2,Tom,10
3,John,20


Here is a visualization to help you understand what `is_john | is_positive` does.

![is_john_OR_is_positive](https://github.com/bdi475/images/blob/main/is-john-OR-is-positive.png?raw=true)


---

**🎯 Example: Find transactions that are NOT made by John**


In [164]:
df = pd.DataFrame(
    {"name": ["John", "Mary", "Tom", "John"], "amount": [-20, -10, 10, 20]}
)

# Create a boolean mask where the name is "John"
is_john = df["name"] == "John"

# Create a boolean mask with the NOT operator
# This will effectively select rows where the name is NOT "John"
df_not_john = df[~is_john]

df_not_john

,name,amount
1,Mary,-10
2,Tom,10


Here's a visualization to help you understand what `~is_john` does.

![not_john](https://github.com/bdi475/images/blob/main/not-john.png?raw=true)


---

## 🗣️ Using Strings for Filtering


Create a new `Series` named `countries`.


In [165]:
countries = pd.Series(
    ["United States", "Oman", "United States", "China", "South Korea", "United States"]
)

display(countries)

0    United States
1             Oman
2    United States
3            China
4      South Korea
5    United States
dtype: object

We can perform element-wise comparisons on string `Series` just like we did with numeric `Series`.

▶️ Compare `countries` with the string `'United States'` using an equality comparison operator (`==`).


In [166]:
countries == "United States"

0     True
1    False
2     True
3    False
4    False
5     True
dtype: bool

▶️ Check the data type of the result.


In [167]:
type(countries == "United States")

pandas.core.series.Series

The result is **another `Series`** containing boolean (`True`/`False`) values. Pandas performs a string comparison (`my_str == 'United States'`) on **each element**.

Remember, you can also supply more than one condition using the following two operators:

1. logical OR (`|`)
2. logical AND (`&`)

▶️ Check whether a country is **either** `'Oman'` **or** `'China'`.


In [168]:
(countries == "Oman") | (countries == "China")

0    False
1     True
2    False
3     True
4    False
5    False
dtype: bool

We can use the resulting boolean `Series` as a mask to filter the original `countries` `Series`.


In [169]:
countries[(countries == "Oman") | (countries == "China")]

1     Oman
3    China
dtype: object

---

### 🔤 Using Strings to Filter a DataFrame


Create a new `DataFrame` named `df_cities`.


In [170]:
df_cities = pd.DataFrame(
    {
        "city": ["Lisle", "Dubai", "Niles", "Shanghai", "Seoul", "Chicago"],
        "country": [
            "United States",
            "United Arab Emirates",
            "United States",
            "China",
            "South Korea",
            "United States",
        ],
        "population": [23270, 3331409, 28938, 26320000, 21794000, 8604203],
    }
)

df_cities

,city,country,population
0,Lisle,United States,23270
1,Dubai,United Arab Emirates,3331409
2,Niles,United States,28938
3,Shanghai,China,26320000
4,Seoul,South Korea,21794000
5,Chicago,United States,8604203


To only keep rows where the `country` is `'United States'`, we can supply a `Series` of boolean values.

Create a new `Series` named `keep` with the following 6 boolean values - `True`, `False`, `True`, `False`, `False`, `True`.


In [171]:
# Manually (not recommended)
keep = pd.Series([True, False, True, False, False, True])

# Programmatically (preferred)
keep = df_cities["country"] == "United States"

keep

0     True
1    False
2     True
3    False
4    False
5     True
Name: country, dtype: bool

To only keep rows where the `"country"` is `"United States"`, use the boolean mask `keep` to filter the `df_cities` `DataFrame`.


In [172]:
df_cities[keep]

,city,country,population
0,Lisle,United States,23270
2,Niles,United States,28938
5,Chicago,United States,8604203


---

**🎯 Example: Cities with population over a million**


In [173]:
df_large_cities = df_cities[df_cities["population"] > 1000000]

df_large_cities

,city,country,population
1,Dubai,United Arab Emirates,3331409
3,Shanghai,China,26320000
4,Seoul,South Korea,21794000
5,Chicago,United States,8604203


---

## 🆙 Sorting a `DataFrame`

You can sort a `DataFrame` using `df.sort_values()`.

![sort_values usage](https://github.com/bdi475/images/blob/main/pandas/sort-values-01.png?raw=true)

The `sort()` method allows you to specify the column(s) to sort by, the sorting order (ascending or descending), and how to handle missing values.

The `by` parameter specifies the column(s) to sort by. You can pass a single column name as a string or multiple column names as a list of strings.

The `ascending` parameter specifies the sorting order. By default, it is set to `True`, which means the data will be sorted in ascending order. If you want to sort in descending order, you can set this parameter to `False`.

If there are multiple columns specified in the `by` parameter, you can pass a list of boolean values to the `ascending` parameter to specify the sorting order for each column individually.

If there are missing values (`NaN`) in the column(s) being sorted, you can use the `na_position` parameter to specify whether to place them at the beginning (`'first'`) or the end (`'last'`) of the sorted `DataFrame`. The default is `'last'`.


▶️ Sort `df_cities` by `population`.


In [174]:
df_cities.sort_values("population")

,city,country,population
0,Lisle,United States,23270
2,Niles,United States,28938
1,Dubai,United Arab Emirates,3331409
5,Chicago,United States,8604203
4,Seoul,South Korea,21794000
3,Shanghai,China,26320000


▶️ Sort `df_cities` by `population` in **descending** order.


In [175]:
df_cities.sort_values("population", ascending=False)

,city,country,population
3,Shanghai,China,26320000
4,Seoul,South Korea,21794000
5,Chicago,United States,8604203
1,Dubai,United Arab Emirates,3331409
2,Niles,United States,28938
0,Lisle,United States,23270


---

## 👟 More Examples using the Yeezys Dataset


In [176]:
df_sneakers = pd.read_csv(
    "https://github.com/bdi475/datasets/raw/main/yeezy_sneakers.csv"
)

# head() displays the first 5 rows of a DataFrame
df_sneakers.head()

,brand,product,price
0,Adidas,Yeezy 750 Boost Light Brown,1578
1,Adidas,Yeezy 350 Boost Pirate Black,910
2,Adidas,Yeezy Boost 350 V2 Lundmark Reflective,1009
3,Adidas,Yeezy 350 Boost V2 Black/Red,954
4,Nike,Air Yeezy Blink,3142


The table below describes the columns in `df_sneakers`.

| Column Name | Description          |
| ----------- | -------------------- |
| brand       | Brand of the sneaker |
| product     | Name of the sneaker  |
| price       | Price of the sneaker |


▶️ Print out a summary of the `df_sneakers` using the `info()` method.


In [177]:
df_sneakers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   brand    15 non-null     object
 1   product  15 non-null     object
 2   price    15 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 492.0+ bytes


▶️ Run `df_sneakers.shape` below to see the _shape_ (number of rows and columns) of the database.


In [178]:
df_sneakers.shape

(15, 3)

---

### 🎯 Challenge 13: Find the number of rows and columns in a `DataFrame`


#### 👇 Tasks

- ✔️ Store the number of rows in `df_sneakers` to a new variable named `num_rows`.
- ✔️ Store the number of columns in `df_sneakers` to a new variable named `num_cols`.
- ✔️ Use `.shape`, not `len()`.


In [179]:
### BEGIN SOLUTION
num_rows = df_sneakers.shape[0]
num_cols = df_sneakers.shape[1]
### END SOLUTION

print(num_rows)
print(num_cols)

15
3


#### 🧭 Check Your Work

- Once you're done, run the code cell below to test correctness.
- ✔️ If the code cell runs without an error, you're good to move on.
- ❌ If the code cell throws an error, go back and fix incorrect parts.


---

**🎯 Example: Find `Adidas` sneakers**


In [180]:
df_adidas = df_sneakers[df_sneakers["brand"] == "Adidas"]

df_adidas

,brand,product,price
0,Adidas,Yeezy 750 Boost Light Brown,1578
1,Adidas,Yeezy 350 Boost Pirate Black,910
2,Adidas,Yeezy Boost 350 V2 Lundmark Reflective,1009
3,Adidas,Yeezy 350 Boost V2 Black/Red,954
10,Adidas,Yeezy Boost 350 V2 Black Reflective,1437
11,Adidas,Yeezy Boost 350 V2 Antlia Reflective,912
12,Adidas,Yeezy Boost 350 V2 Synth Reflective,1292
13,Adidas,Yeezy 350 Boost Turtledove,1279
14,Adidas,Yeezy 750 Boost Glow in the Dark,917


---

**🎯 Example: Find Sneakers under \\$1,000**


In [181]:
df_under_1000 = df_sneakers[df_sneakers["price"] < 1000]

df_under_1000

,brand,product,price
1,Adidas,Yeezy 350 Boost Pirate Black,910
3,Adidas,Yeezy 350 Boost V2 Black/Red,954
11,Adidas,Yeezy Boost 350 V2 Antlia Reflective,912
14,Adidas,Yeezy 750 Boost Glow in the Dark,917


---

**🎯 Challenge 16: Find `Nike` Sneakers over \\$3,000**


In [182]:
df_nike_over_3000 = df_sneakers[
    (df_sneakers["brand"] == "Nike") & (df_sneakers["price"] > 3000)
]

df_nike_over_3000

,brand,product,price
4,Nike,Air Yeezy Blink,3142
5,Nike,Air Yeezy 2 Red October,6075
6,Nike,Air Yeezy 2 Solar Red,4239
7,Nike,Air Yeezy 2 Pure Platinum,3448


---

**🎯 Example: Sort sneakers by price in descending order**


In [183]:
df_sorted_by_price_desc = df_sneakers.sort_values("price", ascending=False)

df_sorted_by_price_desc

,brand,product,price
5,Nike,Air Yeezy 2 Red October,6075
6,Nike,Air Yeezy 2 Solar Red,4239
7,Nike,Air Yeezy 2 Pure Platinum,3448
4,Nike,Air Yeezy Blink,3142
9,Nike,Air Yeezy Zen Grey,2139
8,Nike,Air Yeezy Net,1888
0,Adidas,Yeezy 750 Boost Light Brown,1578
10,Adidas,Yeezy Boost 350 V2 Black Reflective,1437
12,Adidas,Yeezy Boost 350 V2 Synth Reflective,1292
13,Adidas,Yeezy 350 Boost Turtledove,1279


---

**🎯 Example: Sneakers `> 6000` or `< 1000`**


In [184]:
df_polar = df_sneakers[(df_sneakers["price"] > 6000) | (df_sneakers["price"] < 1000)]

df_polar

,brand,product,price
1,Adidas,Yeezy 350 Boost Pirate Black,910
3,Adidas,Yeezy 350 Boost V2 Black/Red,954
5,Nike,Air Yeezy 2 Red October,6075
11,Adidas,Yeezy Boost 350 V2 Antlia Reflective,912
14,Adidas,Yeezy 750 Boost Glow in the Dark,917


---

**🎯 Example: Sort by brand ascending and price descending**


In [185]:
df_sorted_by_brand_price = df_sneakers.sort_values(
    ["brand", "price"], ascending=[True, False]
)

df_sorted_by_brand_price

,brand,product,price
0,Adidas,Yeezy 750 Boost Light Brown,1578
10,Adidas,Yeezy Boost 350 V2 Black Reflective,1437
12,Adidas,Yeezy Boost 350 V2 Synth Reflective,1292
13,Adidas,Yeezy 350 Boost Turtledove,1279
2,Adidas,Yeezy Boost 350 V2 Lundmark Reflective,1009
3,Adidas,Yeezy 350 Boost V2 Black/Red,954
14,Adidas,Yeezy 750 Boost Glow in the Dark,917
11,Adidas,Yeezy Boost 350 V2 Antlia Reflective,912
1,Adidas,Yeezy 350 Boost Pirate Black,910
5,Nike,Air Yeezy 2 Red October,6075
